In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field, replace

import pandas as pd
import numpy as np

from scipy.interpolate import make_splprep
import matplotlib.pyplot as plt
from matplotlib.axes import Axes

import airfoils as afs

from shapely import LineString, Polygon, is_ccw, Point
from shapely import geometry
from shapely.plotting import plot_line, patch_from_polygon
from shapely.affinity import translate, rotate, scale, skew
from shapely import difference, unary_union

from util.path_planning import intro_path
from util.gatekept_itertools import split_indexable, sliding_window
from util.shapely_helpers import split_linestring_by_angle


In [ ]:
@dataclass
class Hole:
    diameter_mm:float
    position:np.ndarray
    def __post_init__(self):
        assert self.position.shape == (2,), "position must have shape `(2)`"

@dataclass
class Airfoil:
    points:np.ndarray
    holes:list[Hole] = field(default_factory=list)

    def __post_init__(self) -> None:
        assert self.points.shape[1]==2, "points must have shape (n, 2)" 
        assert self.points.shape[0]>4, f"need more points. only got {len(self.points)=}"
        assert (self.points[0]==self.points[-1]).all(), "first and last point must be the same"
        assert is_ccw(LineString(self.points)), "Points must be counterclockwise from trailing edge"
    
    @classmethod
    def from_naca(
            cls,
            max_camber:float,
            max_camber_position:float,
            thickness:float,
            chord_length_mm:float
        ):
        upper, lower = afs.gen_NACA4_airfoil(
            max_camber, 
            max_camber_position, 
            thickness,
            200
        )
        selig = np.vstack((upper.transpose()[::-1], lower.transpose()))
        average_terminator = (selig[0]+selig[-1])/2
        selig = np.concatenate([
            [average_terminator],
            selig,
            [average_terminator]
        ])
        return Airfoil(selig*chord_length_mm)
    
    def with_holes(self, holes:list[Hole]) -> Airfoil:
        return replace(self, holes=holes)
    
    def with_translation(self, translation:np.ndarray)->Airfoil:
        assert translation.shape==(2,), "Translation must be an ndarray of shape (2,)"
        return replace(
            self,
            points = self.points+translation.reshape(1,-1),
            holes = [replace(hole,position=hole.position+translation) for hole in self.holes]
        )

    def show(self):
        display(LineString(self.points))
    
    def linestring(self) -> LineString:
        return LineString(self.points)
    
    def polygon(self) -> Polygon:
        return Polygon(self.points)

    def get_with_holes(self, upcut_width=1.3, tolerance=0.05) -> Polygon:
        shapes = [Point(hole.position).buffer(hole.diameter_mm/2) for hole in self.holes]
        height=50
        upcuts = [
            geometry.box(
                *(hole.position-np.array([upcut_width/2,-height])),
                *(hole.position+np.array([upcut_width/2,0]))
            ) for hole in self.holes
        ]
        
        return difference(self.polygon().simplify(tolerance=tolerance),unary_union(shapes+upcuts))
    
    def decompose(self, upcut_kerf=0.1, buffer:float=0, tolerance=0.05, split_angle_deg=75, segment_target_length_or_counts:float|list[float]=1.0):
        lsb = self.get_with_holes(upcut_width=buffer*2+upcut_kerf, tolerance=tolerance)
        if buffer>0:
            lsb = lsb.buffer(buffer, quad_segs=1, join_style="mitre")
        lsb = np.array(lsb.boundary.coords)[:-1]
        
        # change starting point to upper trailing edge
        lsb = np.roll(lsb,-(lsb[:,0]+lsb[:,1]).argmax()-1, axis=0)[::-1]

        # pick out the leading edge point
        leading_edge_split = lsb[:,0].argmin()
        lsbc_upper = lsb[:leading_edge_split+1]
        lsbc_lower = lsb[leading_edge_split:]

        upper_chunks = split_linestring_by_angle(lsbc_upper,split_angle_deg=split_angle_deg)
        lower_chunks = split_linestring_by_angle(lsbc_lower,split_angle_deg=split_angle_deg)

        decomposed = upper_chunks + lower_chunks

        interpolated_chunks = []

        assert isinstance(segment_target_length_or_counts, float) or (isinstance(segment_target_length_or_counts, list) and len(decomposed)==len(segment_target_length_or_counts)), f"segment_length_targets must either be a float or a list of floats the same length as the decomposed line segments: {len(decomposed)}"
        for chunk_index, chunk in enumerate(decomposed):
            if len(chunk)<=2:
                interpolated_chunks.append(chunk)
            else:
                
                if isinstance(segment_target_length_or_counts, float):
                    ls_len = np.linalg.norm(np.diff(chunk, axis=0), axis=1).sum()
                    max_segment_length = segment_target_length_or_counts
                    assert isinstance(max_segment_length, float), f"segment_length_targets must either be a float or a list of floats the same length as the decomposed line segments: {len(decomposed)}"
                    desired_segments = int(np.ceil(ls_len/max_segment_length))
                elif isinstance(segment_target_length_or_counts, list):
                    desired_segments = segment_target_length_or_counts[chunk_index]
                

                bspline, u = make_splprep(chunk.transpose())

                # Evaluate the spline at many points for smooth curve
                u_new = np.linspace(0, 1, desired_segments)
                interpolated_chunks.append(bspline(u_new).transpose())
        return interpolated_chunks
    
    def plot(self, ax:Axes|None=None, decompose_args:None|dict=None):
        if ax is None:
            fig,ax = plt.subplots(figsize=(10,10))
        if decompose_args is None:
            real_decompose_args = {}
        else:
            real_decompose_args = decompose_args
        decomposed = self.decompose(**real_decompose_args)
        for chunk in decomposed:
            ax.plot(*chunk.transpose(),"o-",markersize=2)
        ax.set_aspect("equal")
        return ax, [len(chunk) for chunk in decomposed]
        


In [ ]:
af1 = Airfoil.from_naca(
    max_camber=0.2,
    max_camber_position=0.02,
    thickness=0.13,
    chord_length_mm=150,
).with_holes(
    holes=[
        Hole(5,np.array([20,2]))
    ],
)
af2 = Airfoil.from_naca(
    max_camber=0.2,
    max_camber_position=0.02,
    thickness=0.09,
    chord_length_mm=100,
).with_translation(
    np.array([5,0])
).with_holes(
    holes=[
        Hole(5,np.array([20,2]))
    ],
).with_translation(
    np.array([0,18])
)



In [ ]:
ax, declens = af1.plot()
af2.plot(ax, decompose_args=dict(segment_target_length_or_counts=declens))
chunks_a = af1.decompose()
chunks_b = af2.decompose(segment_target_length_or_counts=[len(chunk) for chunk in chunks_a])
for chunk_a,chunk_b in zip(chunks_a, chunks_b):
    for point_a, point_b in zip(chunk_a, chunk_b):
        ax.plot(*np.stack([point_a, point_b]).transpose())

In [ ]:
np.vstack([point_a, point_b])

In [ ]:
fix, ax = plt.subplots(figsize=(15,10))
ls = af.get_with_holes()
lsb = (
    ls.buffer(0.5, quad_segs=1, join_style="mitre")
    .simplify(tolerance=0.05)
)
ax.add_patch(patch_from_polygon(lsb))
ax.add_patch(patch_from_polygon(ls))
ax.autoscale_view()
ax.set_aspect('equal')

coords = np.array(lsb.boundary.coords)
pnt = coords[(coords[:,0]+coords[:,1]).argmax()]
ax.add_patch(plt.Circle(pnt,0.3, facecolor="red"))

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
decomposed = af.decompose()
for chunk in decomposed:
    ax.plot(*chunk.transpose(),"o-",markersize=2)
ax.set_aspect("equal")

In [ ]:
np.array([np.linalg.norm(np.diff(item)) for item in np.lib.stride_tricks.sliding_window_view(decomposed[0],2,axis=0)]).sum()

In [ ]:
import numpy as np
from scipy.interpolate import make_splprep
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(10,10))
max_segment_length = 1
for chunk in decomposed:
    if len(chunk)<=2:
        ax.plot(*chunk.transpose(), "o-", markersize=0.2)
    else:
        ls_len = np.linalg.norm(np.diff(chunk, axis=0), axis=1).sum()
        desired_segments = int(np.ceil(ls_len/max_segment_length))

        bspline, u = make_splprep(chunk.transpose())

        # Evaluate the spline at many points for smooth curve
        u_new = np.linspace(0, 1, desired_segments)
        x_new, y_new = bspline(u_new)


        ax.plot(x_new, y_new, 'o-',markersize=2)
ax.set_aspect('equal')

In [ ]:

coords = intro_path(np.array(lsb.boundary.coords))
plt.plot(*coords.transpose())
plt.gca().set_aspect("equal")

In [ ]:
pd.DataFrame(coords,columns=["x","y"]).to_parquet("./data/cut_paths/01_100mm_test.parquet")

In [ ]:
lsb = np.array(lsb.boundary.coords)[:-1]
lsb = np.roll(lsb,-(lsb[:,0]+lsb[:,1]).argmax()-1, axis=0)[::-1]
leading_edge_split = lsb[:,0].argmin()
lsbc_upper = lsb[:leading_edge_split]
lsbc_lower = lsb[leading_edge_split:]
fig, ax = plt.subplots()
ax.plot(*lsbc_upper.transpose())
ax.plot(*lsbc_lower.transpose())

In [ ]:
angles = []
lengths = []
for [a,b,c] in sliding_window(lsbc_upper, 3):
    ba = (a-b)/np.linalg.norm(a-b)
    cb = (b-c)/np.linalg.norm(b-c)
    lengths.append(np.linalg.norm(a-b))
    angles.append(np.acos(np.dot(ba, cb)))
pd.Series(angles).plot()

chunks = split_indexable(
    lsbc_upper,
    np.where(abs(np.array(angles))>0.4*np.pi)[0]+1
)

In [ ]:
fig, ax = plt.subplots()
for chunk in chunks+[lsbc_lower]:
    ax.plot(*chunk.transpose())
ax.set_aspect("equal")